# This notebook will try and find an operating point 
# that yields better results for precision

In [ ]:
import csv
import pandas as pd
import numpy as np
import datetime
from datetime import date
import os
pd.set_option('display.max_rows', 1000)
pd.set_option('display.max_columns', 200) 
import matplotlib.pyplot as plt
import seaborn as sns
sns.set(color_codes=True)
from jinja2 import Template
import sys
#reload(sys)
#sys.setdefaultencoding('UTF8')
import matplotlib
import sqlalchemy as sa
#(import database as cc)
%matplotlib inline 
##### Finding tables and columns 
#cc.run_query("""SELECT TOP 10 * from dbc.columnsV""")

In [ ]:
cc.connect()

In [ ]:
#Variables have been replaced.
#This is just an example of how to use lambda if working with a model and needing to
#find a better operating point.

records = cc.run_query("""
    SELECT
    (case when val.id like 'A%' then 'Faculty'
         when val.id like 'B%' then 'Student'
         when val.id like 'C%' then 'Valet' 
         when (val.id not like 'A%' 
               and val.id not like 'B%'
               and val.id not like 'C%') then 'Other' end) as id,
    val.has_car car, 
    val.has_truck truck,
    val.score score,
    val.score_threshold as score_threshold,
    val.grade_decision,
    l.crime_ind,
    l.crime_label as crime_label, 
    l.level_ind as level_ind,
    val.aggregated_decisions_str,
    base.major     
    FROM z.validation_table val
    LEFT JOIN z.labels l
    ON l.id = val.id
    LEFT JOIN z.bases base 
    ON base.id = val.id
    WHERE val.event_dt between '2019-12-01' and '2019-12-31'
    AND val.site in ('USA')
    qualify row_number() over(partition by val.id order by val.event_timestamp DESC)=1
    order by 1 ASC
    """)

In [ ]:
records['score'] = records['score'].astype(float).round(3)
records['score_threshold'] = records['score_threshold'].astype(float)

In [ ]:
def get_precision(df,delta):
    # Creating a temp dataframe that holds modified threshold and desired format
    # Get the total number of records and total flagged count
    # Calculate precision for model:
    # (crime/(crime + non_crimes)) 

    # Find a better operating point for model 
    
    tempdf = (records
              .assign(score_threshold=lambda x: (x['score_threshold'] == delta) + delta))
    num_records = tempdf.shape[0]
    
    num_flagged_df = tempdf[((tempdf.score) > (tempdf.score_threshold))
                       & (tempdf.car == 'true')
                       & (tempdf.truck == 'true')]
    num_flagged = num_flagged_df.shape[0]
    
    person_details = num_flagged_df[(num_flagged_df.id == 'Faculty')]
    person_total_flagged = person_details.shape[0]
         
    crimes = num_flagged_df[(num_flagged_df.id == 'Faculty')]            
    crime_details = crimes[['crime_label']]
    print (crime_details.crime_label.value_counts())    
    
    misdemeanor_df = num_flagged_df[(num_flagged_df.dispatch_id == 'Faculty') 
                           & ((num_flagged_df.crime_label == 'misdemeanor'))]
  
    not_charged_df = misdemeanor_df[(misdemeanor_df.crime_ind == '0')]
    
    federal_df = num_flagged_df[(num_flagged_df.id == 'Faculty') 
                           & ((num_flagged_df.crime_label == 'federal'))]

    charged_df = federal_df[(federal_df.crime_ind == '1')]
    
    misdemeanor_count = misdemeanor_df.shape[0]
    not_charged_count = not_charged_df.shape[0]
    federal_count = federal_df.shape[0]
    charged_count = charged_df.shape[0]

    try:
        precision = (charged_count*100.00)/(charged_count + not_charged_count)
    except ZeroDivisionError:
        precision = 0
  
    return {'Delta': delta,
            'Total_Records':num_records,
            'Misdemeanor_Count': misdemeanor_count, 
            'Federal_Count': federal_count, 
            'person_total_flagged': person_total_flagged,
            'Charged_Count': charged_count, 
            'Not_Charged_Count': not_charged_count,
            'Precision': precision}
    

In [ ]:
results = [get_prec(records, .417)] #for delta in np.arange(0,1,.001)]

In [ ]:
final_results=pd.DataFrame.from_records(results)

In [ ]:
final_results